## check points
1) trading days parity
2) hyperparameter search range(original value in in comment)
3) file path and names

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
# def number_of_years(y):# calculates the number of years of the dataset
#     p = y.index[0]  # date of first row in the dataset (datetime format)
#     q = y.index[len(y) - 1]  # date of last row in the dataset  (datetime format)
#     return ((q - p).days + 1) / 365  # the difference give the number of total days (not trading days) over the total number of years in the dataset
# def number_of_years(y):  # calculates the number of years of the dataset
#     p = y.index[0]  # date of first row in the dataset (datetime format)
#     q = y.index[len(y) - 1]  # date of last row in the dataset  (datetime format)
#     return ((q - p).days + 1) / 365  # the difference give the number of total days (not trading days) over the total number of years in the dataset
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
    weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
    return weights


In [3]:
df = pd.read_csv('n50.csv')

In [4]:
pso_sortino=pd.DataFrame(columns=['Date', 'Returns', 'Risk','Ratio'])
inputweights=[]
trading_days=1

In [5]:
def SORTINO(weights, mean_daily_returns, cov_matrix_neg):
    weights = [w / sum(weights) for w in weights]  # Making sure all weights represent proportions that add up to 1
    weights = np.matrix(weights)
    port_return = np.round(np.sum(weights * mean_daily_returns.T) * trading_days, 2)  # 1259 trading days over 5 year period
    port_semi_dev = np.round(np.sqrt(weights * cov_matrix_neg * weights.T) * trading_days, 2)
    port_semi_dev = float(port_semi_dev)
    sortino_ratio = (port_return - 0.0358/364) / port_semi_dev  # 2.57 represents annual return of risk free security - 5-year US Treasury

    return sortino_ratio


In [6]:
#for lookback
def ratio(a, b, c):
    # function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
    # calculating sharpe ratio
    return (a - c) / b

In [7]:
def PSO_OPTIMIZER(swarm_size, iterations,stock_names,data,dimensions,n50):
    mean_daily_returns = []  # IT WOULD CONTAIN THE AVG RETURS OF THE ASSETS
    all_return_input = []  # cov_input # IT WOULD CONTAIN THE [DAILY RETURNS] OF THE ASSETS
    neg_return = []

    for stock in stock_names:
        indv_stock = data[data.Name == stock]
        avg_return = indv_stock.Daily_returns.mean()
        mean_daily_returns.append(avg_return)
        all_return_input.append(indv_stock.Daily_returns.tolist())

    neg_return = all_return_input[:]
    mean_daily_returns = np.matrix(mean_daily_returns)  # CONVERTING 1D LIST(mean_daily_returns)INTO A MATRIX
    all_return_input = np.matrix(all_return_input)  # CONVERTING A 2D LIST(cov_input)INTO A MATRIX
    #cov_matrix = np.cov(all_return_input)

    for i in range(len(neg_return)):
        for j in range(len(neg_return[i])):
            if neg_return[i][j] > 0:
                neg_return[i][j] = 0
    sortino_input = np.matrix(neg_return)
    cov_matrix_neg = np.cov(sortino_input)

    ''' w,c1,c2 ARE PARAMETERS WHICH WOULD HELP IN CHANGING THE PARTICLE(ASSET) POSITIONS'''
    '''INITIALIZING RANDOM POSITIONS(WEIGHTS) FOR ALL THE 100
    *11 ASSETS'''
    swarm_position = []
    swarm_velocity = []
    for particle in range(swarm_size):  # SWARM SIZE->PORTFOLIO
        '''FOR EACH NEW PORTFOLIO, WE WOULD CREATING A LIST (position)'''
        position = [0] * dimensions
        velocity = [0] * dimensions
        for dimension in range(dimensions):  # DIMENSIONS->ASSET

            position[dimension] = random.random()  # random.random() would assign some random number between 0 and 1
            velocity[dimension] = random.random()
        '''position LIST OF EVERY NEW PORTFOLIO WOULD BE KEPT IN A SEPARATE LIST(swarm_position)'''
        swarm_position.append(position)
        swarm_velocity.append(velocity)

    swarm_position = np.array([np.array(p) for p in swarm_position])  # COVERTING  swarm_position  list to an array
    swarm_velocity = np.array([np.array(v) for v in swarm_velocity])  # COVERTING  swarm_velocities  list to an array

    '''initial_swarm_positions IS A LIST , CONTAINING ALL THE INITIAL POSITIONS OF ALL THE ASSETS OF ALL THE PORTFOLIOS'''
    initial_swarm_positions = swarm_position  # HERE IS THE MASTER FILE WHICH WE WOULD BE USING FOR COMPARISON
    '''swarm_gbest,WE ARE ASSUMING THAT THE FIRST PORTFOLIO IS THE BEST OPTION AVAILABLE '''
    swarm_gbest = initial_swarm_positions[0]

    avg_sortino_list = []
    portfolio_return = []
    #portfolio_standard_deviation = []
    portfolio_sortino = []
    portfolio_semi_deviation = []
    #portfolio_sharpe = []
    portfolio_weights = []

    for iteration in range(iterations):
        '''FOR EACH ITERATION'''
        '''sharpe_pbest_all is a list which would contain the sharpe ratio of the portfolios,Finally at the end it would containing 100 sharpe ratios'''
        sortino_pbest_all = []
        for particle in range(swarm_size):  # SWARM_SIZE->PORTFOLIOS
            '''FOR EACH PORTFOLIO'''
            for dimension in range(dimensions):  # DIMENSIONS->ASSETS
                '''FOR EACH ASSET'''
                '''HERE , WE WOULD BE UPDATING THE POSITIONS AND VELOCITES OF EACH OF THE ASSET PRESENT IN A PARTICULAR PORTFOLIO'''
                r1 = random.random()  # random.random() WOULD BASICALLY GIVE YOU A VALUE BETWEEN 0 AND 1
                r2 = random.random()
                '''HERE WE WOULD BE CHANGING THE PRESENT PARTICLE(ASSET) POSITION 
                   AND FOR CHANGING THE POSITION,WE NEED TO CALCULATE THE NEW VELOCITY'''

                '''BASIC FORMULA 
                   NEW VELOCITY->w*PRESENT VELOCITY + c1 * r1 * abs(INTIAL POSITION - PRESENT POSITION)+c2*r2*abs(GBEST POSITION - PRESENT POSITION)
                   NEW POSITION->PRESENT POSITION + NEW VELOCITY
                '''
                w = (0.9 - 0.4) * ((
                                           iterations - iteration) / iterations) + 0.4  # w at iteration, where initial_w = 0.9 & final_w = 0.4
                c1 = (0.5 - 2.5) * (iteration / iterations) + 2.5  # c1 at iteration, where min_c1 = 0.5 & max_c1 = 2.5
                c2 = (2.5 - 0.5) * (iteration / iterations) + 0.5  # c2 at iteration, where max_c2 = 2.5 & min_c2 = 0.5
                swarm_velocity[particle][dimension] = w * swarm_velocity[particle][dimension] + c1 * r1 * abs(
                    initial_swarm_positions[particle][dimension] - swarm_position[particle][dimension]) + c2 * r2 * abs(
                    swarm_gbest[dimension] - swarm_position[particle][dimension])  # Update velocity in every dimension
                swarm_position[particle][dimension] = swarm_position[particle][dimension] + swarm_velocity[particle][
                    dimension]  # Update position in every direction

            # AFTER 1 COMPLETE PORTFOLIO

            '''AFTER CHANGING THE POSITIONS OF ALL THE ASSETS IN A PARTICULAR PORTFOLIO, WE WOULD FIND THE SHARPE RATIO OF THIS PORTFOLIO
               AND WE WOULD STORE IT IN sharpe_pbest'''
            sortino_pbest = SORTINO(initial_swarm_positions[particle], mean_daily_returns,
                                  cov_matrix_neg)  # Evaluating sharpe of existing pbest position

            '''THEN WE WOULD BE COMPARING THE sharpe_pbest WITH THE SHARPE RATIO OF THE INITIAL PORTFOLIO WHICH IS STORED IN initial_swarm_positions
               ie COMPARING SHARPE RATIO OF THE PORTFOLIO OLD POSITION WITH SHARPE RATIO OF THE PORTFOLIO NEW POSITION'''

            if SORTINO(swarm_position[particle], mean_daily_returns, cov_matrix_neg) > sortino_pbest:
                '''IS THE NEW POSITION BETTER THAN THE EXISTING ONE'''
                '''IF THE NEW POSITION IS BETTER, THEN UPDATE sharpe_pbest  and initial_swarm_positions[particle]'''
                initial_swarm_positions[particle] = swarm_position[particle]  # Update pbest to new position
                sortino_pbest = SORTINO(initial_swarm_positions[particle], mean_daily_returns,
                                      cov_matrix_neg)  # Update sharpe of pbest
            '''IF THE PORTFOLIO WITH NEW POSITION HAS HIGHER SHARPE RATIO THAN
               THE OLD POSITION,APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO NEW POSTION,
               ELSE APPEND SHARPE RATIO OF THE PORTFOLIO WRT TO OLD POSTION '''
            sortino_pbest_all.append(sortino_pbest)
            '''AFTER ALL THE PORTFOLIOS(ie,AFTER ONE ITERATION) ARE DONE, WE WOULD SELECT THE BEST PORTFOLIO
               WRT OF SHARPE RATIO AND COMPARE IT WITH THE SHARPE RATIO OF GLOBAL BEST(swarm_gbest PORTFOLIO)'''

        # AFTER 1 COMPLETE ITERATION

        if max(sortino_pbest_all) > SORTINO(swarm_gbest, mean_daily_returns, cov_matrix_neg):
            ''' IS THE LARGEST SHARPE RATIO OF ALL THE PORTFOLIOS BETTER THEN gbest?'''
            ''' IF YES, CHANGE THE gbest to max(sharpe_pbest_all)'''
            max_index = sortino_pbest_all.index(max(sortino_pbest_all))
            swarm_gbest = initial_swarm_positions[max_index]

        '''AFTER 1 ITERATION IS DONE,ie GOING THROUGH ALL THE PORTFOLIOS AND FINDING THE BEST PORTFOLIO,
           WE WOULD NOW PRINT THE VALUES
        '''
        ''' CALCULATING THE AVG SHARPE RATIO FOR THE ITERATION '''
        avg_sortino = sum(sortino_pbest_all) / len(sortino_pbest_all)

        avg_sortino_list.append(avg_sortino)
        '''NOW FOR PRINTING THE IMPORTANT DATA FROM EACH ITERATION, WE HAVE CREATED A SEPARATE FUNCTION(PRINT_EACH_ITERATION)'''
        # portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation, portfolio_sortino,  portfolio_weights =
        # PRINT_EACH_ITERATION(iteration, swarm_gbest, mean_daily_returns, cov_matrix,cov_matrix_neg, portfolio_return, portfolio_standard_deviation, portfolio_sharpe,portfolio_semi_deviation, portfolio_sortino,  portfolio_weights)

        weights_arr = [w / sum(swarm_gbest) for w in
                       swarm_gbest]  # Making sure all weights represent proportions that add up to 1
        weights = np.matrix(weights_arr)
        port_return = np.round(np.sum(weights * mean_daily_returns.T) * trading_days, 2)   # 1259 trading days over 5 year period
        #port_std_dev = np.round(np.sqrt(weights * cov_matrix * weights.T) * trading_days, 2)
        #port_std_dev = float(port_std_dev)
        #sharpe_ratio = ( port_return - 0.0358/364) / port_std_dev  # 3.58 represents annual return of risk free security - 5-year US Treasury
        port_semi_dev = np.round(np.sqrt(weights * cov_matrix_neg * weights.T) * np.sqrt(trading_days), 2)
        port_semi_dev = float(port_semi_dev)
        sortino_ratio = (port_return - 0.0358/364) / (port_semi_dev)
        portfolio_return.append(port_return)  # Adding portfolio return of a given PORTFOLIO  to  portfolio_return
        #portfolio_standard_deviation.append(port_std_dev)  # Adding portfolio standard deviation of a given PORTFOLIO to a list of standard deviations to portfolio_vol
        #portfolio_sharpe.append(sharpe_ratio)  # Adding portfolio sharpe ratio of a given PORTFOLIO  to   portfolio_sharpe
        portfolio_semi_deviation.append(port_semi_dev)  # Adding portfolio standard deviation of a given PORTFOLIO to a list of standard deviations to portfolio_vol
        portfolio_sortino.append(sortino_ratio)  # Adding portfolio sharpe ratio of a given PORTFOLIO  to   portfolio_sharpe
        portfolio_weights.append(weights_arr)  # Adding portfolio weights of a given PORTFOLIO  to   portfolio_sharpe

    # AFTER ALL THE ITERATIONS
    portfolio = {'Returns' : portfolio_return, 'Semi Deviation' : portfolio_semi_deviation, 'Sortino Ratio' : portfolio_sortino}
    for counter,symbol in enumerate(n50.columns):
        portfolio[symbol + " Weight"] = [Weight[counter] for Weight in portfolio_weights]
    pc = pd.DataFrame(portfolio)
    pc.loc[:, :] *= 100
    pc['Sortino Ratio']=pc['Sortino Ratio']/100 
    max_sortino=pc['Sortino Ratio'].max()     
    max_sortino_portfolio=pc.loc[pc['Sortino Ratio'] == max_sortino]
    max_weights=max_sortino_portfolio.values
    max_weights=max_weights[0][3:]
    return max(portfolio_weights)


In [8]:
def PSO_Sortino(swarm_size,iterations,start_date,end_date):
#     swarm_size=100
#     iterations=5
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')
    df=df.loc[start_date:end_date]
    n50=df
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)
    col = list(df.columns)
    df1 = tdf.reset_index()
    df1 = pd.DataFrame(df1)
    df1 = list(df1.iloc[-1:])[1:]
    stock_names = {}
    for i in range(len(df1)):
        stock_names[df1[i]] = i + 1
    close = []
    name = []
    stock_no = []
    daily_returns = []
    for i in n50:
        t = list(n50[i])  # finding the close price col for each asset
        close.append(t)
        name.append(i)
        t = list(n50[i].pct_change())  # daily return for stock
        daily_returns.append(t)

    closed = []
    for i in close:
        for j in i:
            closed.append(j)
    close = closed

    daily_returns_arr = []

    for i in daily_returns:  # HERE IN THIS CODE ,WE ARE TRYING TO PLACE ALL THE DAILY RETURNS IN A SINGLE LIST Daily_Returnss----><2>
        for j in i:
            daily_returns_arr.append(j)
    daily_returns = daily_returns_arr

    name_arr = []

    x = n50.shape[0]  # FINDING NO OF ROWS, THE REASON IS EACH STOCK IS REPEATING X TIMES

    for i in stock_names:  # ------><3>
        for j in range(x):
            name_arr.append(i)

    name = name_arr
    data = {}
    data['Close'] = close
    data['Daily_returns'] = daily_returns
    data['Name'] = name
    data = pd.DataFrame(data)
    data.isna().sum()
    data = data.fillna(0)  
    dimensions = len(stock_names)  
    avg_sortino_list = []
    portfolio_return = []
    #portfolio_standard_deviation = []
    portfolio_sortino = []
    portfolio_semi_deviation = []
    #portfolio_sharpe = []
    portfolio_weights = []
    portfolio_weights =PSO_OPTIMIZER( swarm_size, iterations,stock_names,data,
                                     dimensions,n50)
#     sharpe_portfolio = {'Returns': portfolio_return, 'Standard Deviation': portfolio_standard_deviation,
#                     'Sharpe Ratio': portfolio_sharpe}
#     for counter, symbol in enumerate(n50.columns):
#         sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in portfolio_weights]
#     sharpe_pc = pd.DataFrame(sharpe_portfolio)
#     sharpe_pc.loc[:, :] *= 100
#     sharpe_pc.loc[:, 'Sharpe Ratio'] /= 100
#     sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
#     sharpe_optimal=sharpe_optimal.to_frame()
#     sharpe_optimal=sharpe_optimal.transpose()
    print(portfolio_weights)
    return portfolio_weights

In [9]:
def PSO_Sortino_lookback(start_date,end_date,deciding_date,pso_sortino,inputweights):
    pso_sortino_list=[]
    df = pd.read_csv('n50.csv', parse_dates=['Date'], index_col='Date')
    df=df.loc[start_date:end_date]
    c=1
    if c==1:
        print(df)
        c+=1
        
    tdf = df.copy()  # deep copy
    df.reset_index(drop=True, inplace=True)
    #trading_days = len(df) / number_of_years(tdf)
    trading_days=1
    returnsh = df.pct_change()  # Here, returnsh would mean return considered for sharpe ratio
    returnsh.fillna(0, inplace=True)  # calculating daily returns of the stocks in the portfolio
    returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
    for cols in returnso.columns.tolist():
        for i in range(0, len(df)):
            if returnso[cols][i] > 0:
                returnso[cols][i] = 0  
    #covmatsh = returnsh.cov() * trading_days  # Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
    covmatso = returnso.cov() * trading_days  # Annualised covariance matrix calculated wrt returnso i.e. used to calculate sortino ratio
    num_assets = len(df.columns)  # initializing number of stocks/assets considered in the portfolio
    risk_free_rate = 0.0358/364  #initializing risk free rate that will be used in calculating both the ratios (absolute value)
# referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
# In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)
    global_war_sortino=[]
    sortino_portfolio_return=[]
    sortino_portfolio_risk=[]
    sortino_portfolio_soratio=[]
    sortino_portfolio_stockWeights=[]
    fitness=0
    weights=np.array(BALANCE(np.array(inputweights)))
    returns_temp = np.sum(returnsh.mean()*weights)*trading_days 
    varso=np.dot(weights.T,np.dot(covmatso,weights))   
    volatility_temp = np.sqrt(varso)      
    fitness = ratio(returns_temp,volatility_temp,risk_free_rate)
    sortino_portfolio_return.append(returns_temp)
    sortino_portfolio_risk.append(volatility_temp)
    sortino_portfolio_soratio.append(fitness)
    sortino_portfolio_stockWeights.append(weights)
    sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  
    for counter,symbol in enumerate(df.columns):
      sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
    sortino_pc = pd.DataFrame(sortino_portfolio)
    sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
    sortino_optimal=sortino_optimal.to_frame()
    sortino_optimal=sortino_optimal.transpose()
    sortino_optimal.loc[:, :] *= 100
    sortino_optimal.loc[:, 'Sortino Ratio'] /= 100
    to_append=[deciding_date,sortino_optimal['Returns'].values[0],sortino_optimal['Semi Deviation'].values[0],sortino_optimal['Sortino Ratio'].values[0]]
    pso_sortino_series = pd.Series(to_append, index = pso_sortino.columns)
    pso_sortino = pso_sortino.append(pso_sortino_series, ignore_index=True)
    sortino_optimal.to_csv(str(deciding_date)+"sortino_PSO_dynamic.csv")
    print(pso_sortino)
    return pso_sortino

In [10]:
start_date="2016-01-01"

In [11]:
start_date_index=df.index[df['Date'] == start_date]

In [12]:
end_date="2020-09-30"

In [13]:
end_date_index=df.index[df['Date']== end_date]

In [14]:
start_date_lookback="2020-10-01"

In [15]:
start_date_lookback_index=df.index[df['Date'] == start_date_lookback]

In [16]:
end_date_lookback="2020-12-31"

In [17]:
end_date_lookback_index=df.index[df['Date']== end_date_lookback]

In [18]:
while df.iloc[end_date_index].Date.values[0]:
    SwarmSize=32
    Iteration=131
    inputweights=PSO_Sortino(SwarmSize,Iteration,df.iloc[start_date_index].Date.values[0],df.iloc[end_date_index].Date.values[0])
    pso_sortino=PSO_Sortino_lookback(df.iloc[start_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index+1].Date.values[0],pso_sortino,inputweights)
    start_date_lookback_index+=1
    end_date_lookback_index+=1
    start_date_index+=1
    end_date_index+=1

[0.0716804552522821, 0.029541708498087266, 0.05192729918421861, 0.04468009175457643, 0.016787770591968794, 0.017356562985889116, 0.017403346590098692, 0.053646185066985506, 0.03352782549063006, 0.052817888849421485, 0.05315080220383386, 0.016784130968719453, 0.03777049750543238, 0.026747399447471433, 0.051484594702235625, 0.048246231205008316, 0.03771800257813868, 0.04728069551692623, 0.059666877788167864, 0.061107241045130996, 0.017024089809216252, 0.06694479777157601, 0.03492043935659984, 0.05178506583738535]
            ADANIPORTS   ASIANPAINT   BAJFINANCE   BAJAJFINSV    BRITANNIA  \
Date                                                                         
2020-10-01  350.842072  2024.912598  3437.245850  6052.132812  3732.739990   
2020-10-05  347.912628  2012.557495  3377.442383  5879.724121  3754.154541   
2020-10-06  360.375305  2074.085205  3468.695068  5993.946777  3690.841309   
2020-10-07  358.140961  2088.971191  3325.775879  5903.818359  3710.541016   
2020-10-08  357

IndexError: positional indexers are out-of-bounds

In [ ]:
# while df.iloc[end_date_index].Date.values[0]:
#     SwarmSize=12
#     Iteration=1
#     inputweights=antcolony_sharpe_weights(ITERATIONS,Q,EVA_RATE,ANTS,df.iloc[start_date_index].Date.values[0],df.iloc[end_date_index].Date.values[0],aco_sharpe)
#     aco_sharpe=ACO_Sharpe_lookback(df.iloc[start_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index].Date.values[0],df.iloc[end_date_lookback_index+1].Date.values[0],aco_sharpe,inputweights)
#     start_date_lookback_index+=1
#     end_date_lookback_index+=1
#     start_date_index+=1
#     end_date_index+=1

In [19]:
pso_sortino.to_csv('pso_sortino_dynamic.csv')